In [6]:
import json

import requests
import pandas as pd
from pathlib import Path

import acs
from censuscodes import county_lookup

In [7]:
# Set up the project resources directory, and the vacancy rates output directory
project_resources_dir = "resources"
project_resources_path = Path(project_resources_dir)

# Set to True to force download from the Census Bureau's API
force_download = False
# include_adjacent_counties = True  # Set to false to limit to NYC boroughs only
include_adjacent_counties = False

In [8]:
# Range of years for the project
project_years = list(range(2012, 2023))

# NYC borough counties
nyc_counties = list(map(county_lookup.by.full_name.get, [
    "Bronx County, New York",
    "Kings County, New York",  # Brooklyn
    "New York County, New York",  # Manhattan
    "Queens County, New York",
    "Richmond County, New York",  # Staten Island
]))

# Counties adjacent to NYC
adjacent_counties = list(map(county_lookup.by.full_name.get, [ 
    "Westchester County, New York",
    "Rockland County, New York",
    "Nassau County, New York",
    "Bergen County, New Jersey",
    "Essex County, New Jersey",
    "Hudson County, New Jersey",
    "Middlesex County, New Jersey",
    "Union County, New Jersey",
    "Fairfield County, Connecticut",
]))

# All counties for the project
project_counties = nyc_counties
if include_adjacent_counties:
    project_counties += adjacent_counties

In [9]:
# Display year and county information for all project years and counties
print(f"Project Years: {project_years}")
print("Project Counties:")
for county in project_counties:
    print(f"- {county.full_name} ("
          f"state_fips={county.state_fips}, "
          f"county_fips={county.fips})")

Project Years: [2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022]
Project Counties:
- Bronx County, New York (state_fips=36, county_fips=005)
- Kings County, New York (state_fips=36, county_fips=047)
- New York County, New York (state_fips=36, county_fips=061)
- Queens County, New York (state_fips=36, county_fips=081)
- Richmond County, New York (state_fips=36, county_fips=085)


In [10]:
# If a csv file exists and we don't want to force a download, load the data
# from the csv file and return it.
csv_path = Path(project_resources_dir) / "vacancy_rates.csv"
if csv_path.exists() and not force_download:
    df = pd.read_csv(csv_path)
else:
    # Otherwise, iterate over the years and counties to get the data, ...
    dfs = []
    for year in project_years:
        for county in project_counties:
            df = acs.get_vacancy_rate(year, county)
            dfs.append(df)

    # ... merge the data, rename and reorder the columns, ...
    df = pd.concat(dfs).reset_index(drop=True)
    df = df[["year", "state", "county", "vacancy_rate"]]
    
    # ... and save the data to a csv file.  Saving the CSV file allows us to
    # avoid making time-consuming API requests to the Census Bureau in the
    # future, as well as merging, column renaming and reordering, etc.
    df.to_csv(csv_path, index=False)

# Display the first few rows of the merged data frame to review the data
df.head()

Year: 2012, County: Bronx County, New York, GET: https://api.census.gov/data/2012/acs/acs1/profile?get=DP04_0001E%2CDP04_0003E&for=county%3A005&in=state%3A36&key=API_KEY_REDACTED
Year: 2012, County: Kings County, New York, GET: https://api.census.gov/data/2012/acs/acs1/profile?get=DP04_0001E%2CDP04_0003E&for=county%3A047&in=state%3A36&key=API_KEY_REDACTED
Year: 2012, County: New York County, New York, GET: https://api.census.gov/data/2012/acs/acs1/profile?get=DP04_0001E%2CDP04_0003E&for=county%3A061&in=state%3A36&key=API_KEY_REDACTED
Year: 2012, County: Queens County, New York, GET: https://api.census.gov/data/2012/acs/acs1/profile?get=DP04_0001E%2CDP04_0003E&for=county%3A081&in=state%3A36&key=API_KEY_REDACTED
Year: 2012, County: Richmond County, New York, GET: https://api.census.gov/data/2012/acs/acs1/profile?get=DP04_0001E%2CDP04_0003E&for=county%3A085&in=state%3A36&key=API_KEY_REDACTED
Year: 2013, County: Bronx County, New York, GET: https://api.census.gov/data/2013/acs/acs1/profile

NameError: name 'merged_df' is not defined